In [1]:
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

DEBUG:root:test


In [2]:
import sys

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

sys.path.insert(0, '/home/gemeinl/code/braindecode-features/')
from braindecode_features import read_features, drop_window, cross_validate

DEBUG:matplotlib:(private) matplotlib data path: /home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:matplotlib data path: /home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/gemeinl/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.3.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'imp

In [3]:
subject_id = 1
train_or_test = 'train'
#path = f'/home/gemeinl/code/mne-features/examples/{subject_id}/{train_or_test}/'
#path = '/home/kiessnek/braindecode_features_master/TUH/train_rec/' 
path = f'./tmp/{train_or_test}'
n_jobs = 2
seed = 20210408
np.random.seed(seed)
only_last_fold = True
# set to integer to enable autosklearn usage with specified integer as training time in minutes
n_min = None  # None/2/20 
drop_a_window = 0

clfs = []
rf = RandomForestClassifier(
    n_estimators=750, 
    random_state=seed
)
clfs.append(rf)
if n_min is not None:
    # optional import which raises warnings
    from autosklearn.experimental.askl2 import AutoSklearn2Classifier
    from autosklearn.classification import AutoSklearnClassifier
    asc = AutoSklearnClassifier( 
        time_left_for_this_task=60*n_min,
        ml_memory_limit=8192,
        n_jobs=n_jobs,
        seed=seed,
        initial_configurations_via_metalearning=0,
    )
    clfs.append(asc)

# read features from disk
df = read_features(
    path=path,
    n_jobs=n_jobs,
)
df[('Description', 'Target')] = df[('Description', 'Target')].astype('int64')

# potentially drop first window (for HGD, better decoding when starting at 500ms?)
if drop_a_window is not None:
    df = drop_window(df, drop_a_window)

# try different feature usages
# TODO: fix None, False. is problematic with variable length trials
agg_funcs = [None, np.mean]
windows_as_examples_ = [True, False]
all_results = []
for clf in clfs:
    # TODO: add filter_df of feature dataframe
    for agg_func, windows_as_examples in zip(agg_funcs, windows_as_examples_):
        results = cross_validate(
            df=df,
            clf=clf,
            subject_id=subject_id,
            only_last_fold=only_last_fold,
            agg_func=agg_func,
            windows_as_examples=windows_as_examples,
            out_path=None,
        )
        all_results.append(results)
all_results = pd.concat(all_results)

/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2876: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:braindecode_features.decoding:train shapes (1325, 1092), (1325,), (1325,)
DEBUG:braindecode_features.decoding:valid shapes (225, 1092), (225,), (225,)
INFO:braindecode_featur

In [4]:
all_results

,subject,fold,estimator,window_accuracy,trial_accuracy,predictions,targets,model,feature_names,windows_as_examples,agg_func
0,1,4,RandomForestClassifier,0.0,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","RandomForestClassifier(bootstrap=True, ccp_alp...","{'Domain': {0: 'Time', 1: 'Time', 2: 'Time', 3...",True,None
0,1,4,RandomForestClassifier,1.0,1.0,[1],[1],"RandomForestClassifier(bootstrap=True, ccp_alp...","{'Domain': {0: 'Time', 1: 'Time', 2: 'Time', 3...",False,mean


In [5]:
df

Domain    Description                     Time                        \
Feature         Trial Window Target covariance                         
Channel                                  A1-A2      A1-C3      A1-C4   
Frequency                                  4-8        4-8        4-8   
1                   0      0      0  26.628353  10.104492  10.887094   
2                   0      1      0  29.428747  10.640343   9.050004   
3                   0      2      0  96.053429  40.247276  26.349131   
4                   0      3      0  72.775475  35.462242  26.417665   
5                   0      4      0  21.345385   9.267915   5.860884   
...               ...    ...    ...        ...        ...        ...   
1550                4    266      1   4.301175   1.253188   1.083367   
1551                4    267      1   5.686838   0.934893   1.187841   
1552                4    268      1   3.335802   0.620052   0.467795   
1553                4    269      1   4.507133   1.057006   1.334987   
1554                4    270      1   6.496952   2.568326  -1.184212   

Domain                                                 ...  \
Feature                                                ...   
Channel        A1-CZ      A1-F3      A1-F4      A1-F7  ...   
Frequency        4-8        4-8        4-8        4-8  ...   
1           4.587402   8.121691  10.065985  21.740244  ...   
2           2.758714   4.395134   6.251713  16.526342  ...   
3           8.243728  27.083361  21.236219  60.277660  ...   
4          11.999883  17.837280  12.439607  43.149849  ...   
5           6.213284   3.727525   3.304119  13.120700  ...   
...              ...        ...        ...        ...  ...   
1550       -0.169095   1.410296   0.913391   3.592587  ...   
1551       -0.082926   1.206407   1.171357   5.216403  ...   
1552        0.117123  -0.022159   0.167063   1.917980  ...   
1553       -0.015315   1.817538   0.911703   4.681142  ...   
1554       -1.672123   0.262131  -8.617229  14.736306  ...   

Domain                                                                         \
Feature   zero_crossings_derivative                                             
Channel                          FZ     O1     O2     P3     P4     PZ     T1   
Frequency                      8-13   8-13   8-13   8-13   8-13   8-13   8-13   
1                             115.0  106.0  107.0  100.0  110.0  107.0  108.0   
2                             104.0   97.0  101.0   97.0  101.0  103.0  101.0   
3                             104.0   95.0   90.0   85.0   90.0   87.0   95.0   
4                              99.0   92.0   91.0   92.0   91.0   89.0  101.0   
5                             109.0  108.0  106.0  109.0  106.0  106.0  110.0   
...                             ...    ...    ...    ...    ...    ...    ...   
1550                          115.0  106.0  109.0  112.0  111.0  104.0  108.0   
1551                          106.0  103.0  101.0  108.0  113.0  110.0  100.0   
1552                          114.0  119.0  122.0  121.0  120.0  120.0  120.0   
1553                          119.0  115.0  115.0  118.0  115.0  119.0  101.0   
1554                          117.0  111.0  112.0  115.0  115.0  117.0  103.0   

Domain                          
Feature                         
Channel       T2     T3     T4  
Frequency   8-13   8-13   8-13  
1          103.0  108.0  103.0  
2          101.0  101.0  105.0  
3           94.0   97.0   96.0  
4           99.0   96.0  101.0  
5          114.0  111.0  108.0  
...          ...    ...    ...  
1550       109.0  104.0  109.0  
1551       107.0  108.0  115.0  
1552       116.0  113.0  118.0  
1553       107.0  111.0  109.0  
1554       109.0  109.0  111.0  

[1550 rows x 1095 columns]

In [5]:
ls

before.csv                             braindecode_features_extract.ipynb
braindecode_features_decode.ipynb      braindecode_features_extract_old.ipynb
braindecode_features_decode_old.ipynb  tmp/


In [3]:
results = pd.read_csv(
    '/home/gemeinl/code/mne-features/examples/1/cv_results.csv',
    index_col=0
).reset_index(drop=True)

In [4]:
results.columns

Index(['subject', 'fold', 'estimator', 'window_accuracy', 'trial_accuracy',
       'predictions', 'targets', 'model', 'feature_names',
       'windows_as_examples', 'agg_func'],
      dtype='object')

In [6]:
results[['subject', 'estimator', 'window_accuracy', 'trial_accuracy', 'agg_func', 'windows_as_examples']]

,subject,estimator,window_accuracy,trial_accuracy,agg_func,windows_as_examples
0,1,RandomForestClassifier,0.753125,0.859375,NaN,True
1,1,RandomForestClassifier,0.921875,0.921875,NaN,False
2,1,RandomForestClassifier,0.906250,0.906250,mean,False
